# Neural Networks and Deep Learning - Lab 4

This notebook solves both problems mentioned in **Lab #4** using TensorFlow and NumPy.

---

## ✅ Program #1: House Price Prediction (Regression)

We'll create a synthetic version of the `Houseprice_Bangalore` dataset and use a neural network to predict house prices. We'll train using:

- Batch Gradient Descent
- Stochastic Gradient Descent (SGD)
- Mini-Batch Gradient Descent

Metric: Mean Squared Error (MSE)


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
tf.random.set_seed(42)

In [ ]:
# Simulate dataset
np.random.seed(0)
num_samples = 1000

# Features: bedrooms, size (sqft), age, bathrooms
X = np.random.randint(1, 6, size=(num_samples, 4)).astype(float)
X[:, 1] *= 500  # size in sqft
X[:, 2] *= 5    # age in years

# Price formula (just for simulation)
y = 50000 + (X[:, 0] * 100000) + (X[:, 1] * 300) - (X[:, 2] * 1000) + (X[:, 3] * 50000)
y = y.reshape(-1, 1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(16, activation='relu', input_shape=(4,)),
        tf.keras.layers.Dense(1)
    ])
    return model


In [ ]:
optimizers = {
    'Batch Gradient Descent': tf.keras.optimizers.SGD(learning_rate=0.01),
    'Stochastic Gradient Descent': tf.keras.optimizers.SGD(learning_rate=0.01),
    'Mini-Batch Gradient Descent': tf.keras.optimizers.SGD(learning_rate=0.01)
}

batch_sizes = {
    'Batch Gradient Descent': len(X_train),
    'Stochastic Gradient Descent': 1,
    'Mini-Batch Gradient Descent': 32
}

results = {}

for name in optimizers:
    print(f"Training with {name}")
    model = build_model()
    model.compile(optimizer=optimizers[name], loss='mse')
    model.fit(X_train, y_train, epochs=50, batch_size=batch_sizes[name], verbose=0)
    preds = model.predict(X_test)
    mse = mean_squared_error(y_test, preds)
    results[name] = mse
    print(f"{name} - Test MSE: {mse:.2f}\n")


---

## ✅ Program #2: Single-Layer NN Alarm System (Manual Backpropagation)

We will manually implement a single-layer neural network with 2 inputs (`x₁`, `x₂`) and one output using sigmoid activation. We'll perform:

- Forward pass
- Backward pass (backpropagation using delta rule)
- Update weights & bias


In [ ]:
# Inputs and expected outputs (truth table)
X = np.array([[0,0], [0,1], [1,0], [1,1]])
y = np.array([[0], [1], [1], [1]])  # Alarm triggers if any sensor is on

# Sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivative of sigmoid
def sigmoid_deriv(x):
    return x * (1 - x)

# Initialize weights and bias
np.random.seed(1)
weights = np.random.rand(2,1)
bias = np.random.rand(1)
lr = 0.1

# Forward pass
z = np.dot(X, weights) + bias
y_pred = sigmoid(z)

# MSE before update
error = y - y_pred
mse_before = np.mean(error**2)

# Backpropagation
d_pred = error * sigmoid_deriv(y_pred)
weights_update = lr * np.dot(X.T, d_pred)
bias_update = lr * np.sum(d_pred)

# Update
weights += weights_update
bias += bias_update

# Final weights and bias
print("MSE before update:", round(mse_before, 4))
print("Updated Weights:\n", weights)
print("Updated Bias:\n", bias)
